In [ ]:
import torch
import gym
import gymu
import numpy as np
from itertools import chain, repeat

class TestEnvironment(gym.Env):
    
    def __init__(self, n=10, m=2):
        shape = (1,2,2)
        self.states = torch.ones(n+1,*shape) * (torch.arange(n+1).view(n+1,1,1,1) / n)
        self.action_space = gym.spaces.Discrete(m)
        self.observation_space = gym.spaces.Box(0,1,shape)
        self.index = 0
        self.count = 0
        
    def step(self, action):
        self.count += 1
        self.index += action
        self.index = min(self.index, self.states.shape[0] - 1)
        done = self.index == (self.states.shape[0] - 1)
        return self.states[self.index], 0., done, {'index':self.index, 'step':self.count}
    
    def reset(self):
        self.index = 0
        self.count = 0
        return self.states[self.index], {'index':self.index, 'step':self.count}
    
env = TestEnvironment()
iterator = gymu.iterator(env, mode=gymu.mode.sardi)

iterator = chain(*list(repeat(iterator, 2)))

default = next(iter(gymu.data.dataset(gymu.iterator(env, mode=gymu.mode.sardi)).gymu.unpack_info()))
default['step'] = -1

dataset = gymu.data.dataset(iterator).gymu.unpack_info()

mask = {'state': np.s_[:2] }
dataset = dataset.gymu.window(window_size=10, default=default, mask=mask)
#dataset = dataset.gymu.window(window_size=2)

for x in dataset:
    print(x['state'].shape, x['action'].shape)
    print(x['step'])
    
    




        

In [4]:
import gymu
import glob

env = gymu.make("thesis/MNIST-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
gymu.data.write_episode(iterator, path=f"./MNIST-v0/ep-{0}")
episodes = glob.glob("./MNIST-v0/*.tar.gz")
dataset = gymu.data.dataset(episodes)
dataset = dataset.decode(keep_meta=False)
dataset = dataset.window(window_size=3, default='zeros_like')
dataset = dataset.mask(state = slice(-1))
dataset = dataset.numpy()
dataset = dataset.mode(gymu.mode.sa)
dataset = dataset.to_tensor_dataset(show_progress=True, order=['state', 'action'])

import torch
for s,a in iter(dataset):
    print(type(s), type(a))
#loader = DataLoader(dataset, batch_size=2)
import jnu as J
for x in iter(dataset):
    s,a = x
    J.image(torch.cat([*s], dim=-1))
    print(s.shape, a)
    #print(x['__key__'], x['__worker__'])

0it [00:00, ?it/s]

THEN
THEN
THEN
THEN
THEN


Loading Tensor Dataset: 0it [00:00, ?it/s]

[<class 'torch.Tensor'>, <class 'torch.Tensor'>]
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>
<class 'torch.Tensor'> <class 'torch.Tensor'>


torch.Size([2, 1, 28, 28]) tensor([0, 0, 1])


...

torch.Size([2, 1, 28, 28]) tensor([0, 0, 0])


torch.Size([2, 1, 28, 28]) tensor([0, 0, 1])


torch.Size([2, 1, 28, 28]) tensor([0, 1, 0])


torch.Size([2, 1, 28, 28]) tensor([1, 0, 1])


torch.Size([2, 1, 28, 28]) tensor([0, 1, 1])


In [ ]:
import torch
import numpy as np
torch.cat([[torch.zeros(10)], [torch.zeros(10)]])

In [ ]:
np.array(np.zeros(10)).shape

In [52]:
import re

pattern = re.compile("\[(-?[0-9]*):(-?[0-9]*):?(-?[0-9]*)|(-?[0-9]*)\]")
print(re.findall(pattern, "[0]"))
print(re.findall(pattern, "[:]"))
print(re.findall(pattern, "[1:]"))
print(re.findall(pattern, "[:-1]"))
print(re.findall(pattern, "[::-1]"))
print(re.findall(pattern, "[1::-1]"))
print(re.findall(pattern, "[1:2]"))
print(re.findall(pattern, "[1:2:-1]"))


s = [(int(x) if x != '' else None) for x in re.findall(pattern, "[1:2:-1]")[0]]
s = s[-1] if s[-1] is not None else s[:3]
print(s)

[('', '', '', '0')]
[('', '', '', ''), ('', '', '', '')]
[('1', '', '', ''), ('', '', '', '')]
[('', '-1', '', ''), ('', '', '', '')]
[('', '', '-1', ''), ('', '', '', '')]
[('1', '', '-1', ''), ('', '', '', '')]
[('1', '2', '', ''), ('', '', '', '')]
[('1', '2', '-1', ''), ('', '', '', '')]
[1, 2, -1]
